In [30]:
import datetime
from hashlib import sha256

BLOCK_MAX_SIZE = 5
def isCryptographicPuzzleSolved(aBlock):
    return aBlock.hash() < (2**256) / aBlock.header['difficulty'] - 1
    
class Block: 
    def __init__(self, transactions):
        self.header = {
            'prev_hash': 0,
            'nonce': 0,
            'timestamp': None,
            'transactions_amount': len(transactions),
            'difficulty': 1
        }
        if (len(transactions) <= BLOCK_MAX_SIZE):
            self.transactions = transactions
        else:
            raise 'Exceeding max block size'
            
    def hash(self):
        return int(sha256(repr(self.header).encode('utf-8') + repr(self.transactions).encode('utf-8')).hexdigest(), 16)
        
    def __str__(self):
        transactions = ",".join(self.transactions)
        return """
        'block_hash': {0}
        
        'header': {{
            'prev_hash':{1}
            'nonce': {2}
            'timestamp': {3}
            'transactions_amount': {4}
            'difficulty': {5}
        }}
        
        'transactions': [
            {6}
        ]
        """.format(hex(self.hash()), hex(self.header['prev_hash']), self.header['nonce'], self.header['timestamp'], self.header['transactions_amount'], self.header['difficulty'], transactions)
    
class Blockchain:
    def __init__(self):
        self.blocks = []
        self.last_block_hash = 0
        
    def addBlock(self, newBlock):
        if (self.isBlockValid(newBlock)):
            self.blocks.append(newBlock)
            self.last_block_hash = newBlock.hash()
            return True
        return False
    
    def isBlockValid(self, block):
        return block.header['prev_hash'] == self.last_block_hash and isCryptographicPuzzleSolved(block)
    
    def getLastHash(self):
        return self.last_block_hash
    
    def printBlockChain(self):
        for block in self.blocks:
            print ('-----------------------------------------------------------------------------')
            print (block)
            print ('-----------------------------------------------------------------------------')

In [8]:
def mine(prev_hash, block):
    block.header['prev_hash'] = prev_hash
    block.header['timestamp'] = datetime.datetime.now()
    while not isCryptographicPuzzleSolved(block):
        block.header['nonce'] += 1
        block.header['timestamp'] = datetime.datetime.now()

In [9]:
# Example creating a blockchain and adding one block only. 
# The second block doesn't get added because it does not match the prev_hash
coolBlockChain = Blockchain()
newBlock = Block(['transaction_1', 'transaction_2'])

mine(coolBlockChain.getLastHash(), newBlock)

coolBlockChain.addBlock(newBlock)
coolBlockChain.addBlock(newBlock)

In [10]:
coolBlockChain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0x5c076f392681b435a43669f31542d619c3c4b1c1d71f8c8986160c11708a9c13
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 18:10:16.502769
            'transactions_amount': 2
            'difficulty': 1
        }
        
        'transactions': [
            transaction_1,transaction_2
        ]
        
-----------------------------------------------------------------------------


In [514]:
def adjustDifficulty(prevDifficulty, elapsedTime, targetTime, blocksProcessed):
    return prevDifficulty * (blocksProcessed/elapsedTime)*targetTime

In [515]:
def generateBlock(blockchain, difficulty, i):
    newBlock = Block(['transaction_{0}'.format(i)])
    newBlock.header['difficulty'] = difficulty
    mine(blockchain.getLastHash(), newBlock)
    blockchain.addBlock(newBlock)
    
    print ('{0} | Block {2} mined with difficulty: {1}!'.format(datetime.datetime.now(), difficulty, i))
    print (newBlock)

In [516]:
# Example of how difficulty is adjusted.
TARGET_TIME_IN_SECONDS = 60 # Every 60s a new block is mined in avg.
BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY = 256 # Every 256 blocks difficulty is adjusted to match TARGET_TIME.

elapsedTimeInSeconds = 0
blocksProccesed = 0

awesomeBlockchain = Blockchain()
difficulty = 1

startTime = datetime.datetime.now()

genesisBlock = Block(['transaction_genesis'])
mine(awesomeBlockchain.getLastHash(), genesisBlock)
awesomeBlockchain.addBlock(genesisBlock)

blocksProccesed += 1
i = 0
# Simulation of a working blockchain.
# A random block is generated in each iteration and difficulty is adjusted dynamically
while len(awesomeBlockchain.blocks) < 10000:
    if blocksProccesed >= BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY:
        elapsedTimeInSeconds = (datetime.datetime.now() - startTime).total_seconds()
        difficulty = adjustDifficulty(difficulty, elapsedTimeInSeconds, TARGET_TIME_IN_SECONDS, blocksProccesed)
        blocksProccesed = 0
        startTime = datetime.datetime.now()
        
    generateBlock(awesomeBlockchain, difficulty, i)
    i += 1
    blocksProccesed += 1

2021-04-26 22:14:40.778914 | Block 0 mined with difficulty: 1!

        'block_hash': 0x8e5285a8a9479c949d069d182be40580393635d7d91fe0dec5b416ec5a9cc01c
        
        'header': {
            'prev_hash':0x6aacb1318dd81115796c135513e1dffacb6feae6b5f7f335f2aed66fbb2efa37
            'nonce': 0
            'timestamp': 2021-04-26 22:14:40.778914
            'transactions_amount': 1
            'difficulty': 1
        }
        
        'transactions': [
            transaction_0
        ]
        
2021-04-26 22:14:40.779914 | Block 1 mined with difficulty: 1!

        'block_hash': 0x5ffc3280682616db4617a2cc427c9c98a2efc5ec041cd967d64822136fb425e
        
        'header': {
            'prev_hash':0x8e5285a8a9479c949d069d182be40580393635d7d91fe0dec5b416ec5a9cc01c
            'nonce': 0
            'timestamp': 2021-04-26 22:14:40.779914
            'transactions_amount': 1
            'difficulty': 1
        }
        
        'transactions': [
            transaction_1
        ]
    

2021-04-26 22:14:41.341422 | Block 255 mined with difficulty: 74101.59057906341!

        'block_hash': 0x58f28e179875ee2d44953b8c29090d80443ae6540996b29293ddcef2a1d
        
        'header': {
            'prev_hash':0x6307e044a29f73c95d8de4ff56975f10315834cbef7e27811632d7a69ccb276a
            'nonce': 26123
            'timestamp': 2021-04-26 22:14:41.341422
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_255
        ]
        
2021-04-26 22:14:43.881166 | Block 256 mined with difficulty: 74101.59057906341!

        'block_hash': 0x19dda67f68a9532311bfc498cd15402b7ea7e0be10a25a72e2970b0dac54
        
        'header': {
            'prev_hash':0x58f28e179875ee2d44953b8c29090d80443ae6540996b29293ddcef2a1d
            'nonce': 189026
            'timestamp': 2021-04-26 22:14:43.881166
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        


2021-04-26 22:14:54.366486 | Block 272 mined with difficulty: 74101.59057906341!

        'block_hash': 0xce4358f99b01443aaf7aa6aded40ad3b36d94178f05aaf5053a2043786fc
        
        'header': {
            'prev_hash':0x220e686d1c39a7f972e316fea91fc96d237fc94037f07841943ac261fde0
            'nonce': 33510
            'timestamp': 2021-04-26 22:14:54.366486
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_272
        ]
        
2021-04-26 22:14:55.272060 | Block 273 mined with difficulty: 74101.59057906341!

        'block_hash': 0x48b6e47043552c9e2e59a84d3e2d3c44d056311b93d4286635c6977d60c5
        
        'header': {
            'prev_hash':0xce4358f99b01443aaf7aa6aded40ad3b36d94178f05aaf5053a2043786fc
            'nonce': 70693
            'timestamp': 2021-04-26 22:14:55.272060
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
   

2021-04-26 22:15:11.885152 | Block 289 mined with difficulty: 74101.59057906341!

        'block_hash': 0x6e664660c441d37478ccf9d53e10f5591f2f36ca705dadd12688643483a8
        
        'header': {
            'prev_hash':0x20615fc0a430ba61193c06e158fc3a5c45e66f8440bc69597014342ae3ea
            'nonce': 168655
            'timestamp': 2021-04-26 22:15:11.885152
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_289
        ]
        
2021-04-26 22:15:14.355810 | Block 290 mined with difficulty: 74101.59057906341!

        'block_hash': 0x6c8f164113183a7b9894f7a880c01b33829c43c19aa60a3fa6570c41b081
        
        'header': {
            'prev_hash':0x6e664660c441d37478ccf9d53e10f5591f2f36ca705dadd12688643483a8
            'nonce': 196633
            'timestamp': 2021-04-26 22:15:14.355810
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
 

2021-04-26 22:15:28.450320 | Block 305 mined with difficulty: 74101.59057906341!

        'block_hash': 0x2f183a7e26732b53279b76601aaaf0ee88855f84d2837b678d38b8763703
        
        'header': {
            'prev_hash':0x2b3e820b027569bb859b6e258da86d67eb0e8d7fd9b32cc5de11b5de6e98
            'nonce': 15355
            'timestamp': 2021-04-26 22:15:28.449320
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_305
        ]
        
2021-04-26 22:15:29.407075 | Block 306 mined with difficulty: 74101.59057906341!

        'block_hash': 0x3551d574b47b4f693f908a02e43f42fc0fbb05d67dd568bba63abbce901e
        
        'header': {
            'prev_hash':0x2f183a7e26732b53279b76601aaaf0ee88855f84d2837b678d38b8763703
            'nonce': 54384
            'timestamp': 2021-04-26 22:15:29.407075
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
   

...

2021-04-26 22:17:53.655994 | Block 452 mined with difficulty: 74101.59057906341!

        'block_hash': 0x716ccee3e0c67e004a25ddfae2b5e63580a34d967f88cf061273815250e8
        
        'header': {
            'prev_hash':0x9f74f52eac083393dfe8aca153e2f967ebf72600de898beeaade9bcacc5
            'nonce': 22094
            'timestamp': 2021-04-26 22:17:53.655994
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_452
        ]
        
2021-04-26 22:17:53.858038 | Block 453 mined with difficulty: 74101.59057906341!

        'block_hash': 0xc82cafe67478439a077e5cee560ae98156c57764c18f2b3332037efead
        
        'header': {
            'prev_hash':0x716ccee3e0c67e004a25ddfae2b5e63580a34d967f88cf061273815250e8
            'nonce': 16577
            'timestamp': 2021-04-26 22:17:53.857036
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
      

2021-04-26 22:18:13.836226 | Block 470 mined with difficulty: 74101.59057906341!

        'block_hash': 0xa928d4ea02784c3d2b52e46f7d3e9cf13d9d038d73d651e1a1e799359960
        
        'header': {
            'prev_hash':0x51f685b8c2fe82e21fca006e82ceddd3bdd9e55cd6d0056610a122d4dd7e
            'nonce': 235405
            'timestamp': 2021-04-26 22:18:13.836226
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_470
        ]
        
2021-04-26 22:18:14.647760 | Block 471 mined with difficulty: 74101.59057906341!

        'block_hash': 0x11a89a7b1aa99bd07fe1f728c575fd5c0f136f6b719ca2a543f68aea17c
        
        'header': {
            'prev_hash':0xa928d4ea02784c3d2b52e46f7d3e9cf13d9d038d73d651e1a1e799359960
            'nonce': 55199
            'timestamp': 2021-04-26 22:18:14.647760
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
   

2021-04-26 22:18:32.289610 | Block 486 mined with difficulty: 74101.59057906341!

        'block_hash': 0x57bd39926d07bb008ac2e9bee2aa4f80226d824cc2cc4f5d9d181894367e
        
        'header': {
            'prev_hash':0x369f86b39d9e499fe8874a3c79201c41fa51af06e6fa99db952159a6f147
            'nonce': 38075
            'timestamp': 2021-04-26 22:18:32.289610
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_486
        ]
        
2021-04-26 22:18:32.327606 | Block 487 mined with difficulty: 74101.59057906341!

        'block_hash': 0x8fcb809ef56307037f1f571aec879f5568371cc083eb1dbae5bb1f0009c3
        
        'header': {
            'prev_hash':0x57bd39926d07bb008ac2e9bee2aa4f80226d824cc2cc4f5d9d181894367e
            'nonce': 2583
            'timestamp': 2021-04-26 22:18:32.327606
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
    

2021-04-26 22:18:44.727124 | Block 502 mined with difficulty: 74101.59057906341!

        'block_hash': 0x5b2505ff36f660b19fea6c3eab11e1a76e3773a4cfbeab07b52349297b5e
        
        'header': {
            'prev_hash':0xe03a04bb1730179bca56debaf83b951dd09633576fd759e7c0d9fb98cf42
            'nonce': 25375
            'timestamp': 2021-04-26 22:18:44.727124
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
        'transactions': [
            transaction_502
        ]
        
2021-04-26 22:18:44.947362 | Block 503 mined with difficulty: 74101.59057906341!

        'block_hash': 0x22189715470638214cbe5f9e9df027f987b1b2a93c18ffbd83a6f29ef53f
        
        'header': {
            'prev_hash':0x5b2505ff36f660b19fea6c3eab11e1a76e3773a4cfbeab07b52349297b5e
            'nonce': 17739
            'timestamp': 2021-04-26 22:18:44.947362
            'transactions_amount': 1
            'difficulty': 74101.59057906341
        }
        
   

KeyboardInterrupt: 

In [11]:
# Example on how multiple clients issue transactions to the system and those are batched together in blocks with max size 5.

def receiveTransactionFromClient(i):
    return 'transaction_{0}'.format(i)

amazingBlockchain = Blockchain()
difficulty = 1

genesisBlock = Block(['transaction_genesis'])
mine(amazingBlockchain.getLastHash(), genesisBlock)
amazingBlockchain.addBlock(genesisBlock)

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), firstBlock)
amazingBlockchain.addBlock(firstBlock)

transactions = []
# Receive 3 transactions and batch them together.
transactions = []
for i in range(3):
    transactions.append(receiveTransactionFromClient(i))

secondBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), secondBlock)
amazingBlockchain.addBlock(secondBlock)

amazingBlockchain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0x9693d040b9162de4973c849ef817c4c1c1bacc793fad22fa36427628abb629d9
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 18:10:19.987838
            'transactions_amount': 1
            'difficulty': 1
        }
        
        'transactions': [
            transaction_genesis
        ]
        
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

        'block_hash': 0xab5635258232406e86699a3458cdadc7c8a7e0b418780bee8d51afbe5be16264
        
        'header': {
            'prev_hash':0x9693d040b9162de4973c849ef817c4c1c1bacc793fad22fa36427628abb629d9
            'nonce': 0
            'timestamp': 2021-04-27 18:10:19.987838
            'transactions_amount': 5
            'difficulty': 1
        }
        
        'trans

In [39]:
# Example dispatching a block to be mine by multiple workers using multithreading module.
import threading
import copy

runningOutOfNamesBlockchain = Blockchain()

def mineConcurrent(prev_hash, block, blockchain, i):
        block.header['difficulty'] = 100000
        block.header['nonce'] = i**7
        block.header['prev_hash'] = prev_hash
        block.header['timestamp'] = datetime.datetime.now()
        while not isCryptographicPuzzleSolved(block):
                block.header['nonce'] += 1
                block.header['timestamp'] = datetime.datetime.now()
        if (blockchain.addBlock(block)):
            print ('Block mined by thread {0}'.format(i))

def dispatchBlock(W, block):
    threads = []
    block_mined = None
    last_hash = runningOutOfNamesBlockchain.getLastHash()
    for i in range(W):
        t = threading.Thread( target=mineConcurrent, args=(last_hash, copy.deepcopy(firstBlock), runningOutOfNamesBlockchain, i) )
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)

# Dispatch the block to 3 workers to mine the work.
dispatchBlock(3, firstBlock)

runningOutOfNamesBlockchain.printBlockChain()


Block mined by thread 0
-----------------------------------------------------------------------------

        'block_hash': 0xa0c8b63e76673c3e0126a3b7bc482eefbacbb4fd86c9969558781f9657e7
        
        'header': {
            'prev_hash':0x0
            'nonce': 35840
            'timestamp': 2021-04-27 18:34:28.423882
            'transactions_amount': 5
            'difficulty': 100000
        }
        
        'transactions': [
            transaction_0,transaction_1,transaction_2,transaction_3,transaction_4
        ]
        
-----------------------------------------------------------------------------
